<a href="https://colab.research.google.com/github/artakhm/HomeCredit/blob/master/table_agg_hame_credit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
sns.set(style="darkgrid")

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from google.colab import drive  
drive.mount('/content/drive')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Mounted at /content/drive


In [ ]:
description = pd.read_csv('/content/drive/My Drive/HomeCredit/HomeCredit_columns_description.csv', encoding = "ISO-8859-1", index_col='Unnamed: 0')

In [ ]:
description.head()

,Table,Row,Description,Special
1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN


## APPLICATION TRAIN

In [ ]:
train = pd.read_csv('/content/drive/My Drive/HomeCredit/application_train.csv')

In [ ]:
train["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

In [ ]:
train= train.fillna(train.median())
train = train.apply(lambda x:x.fillna(x.value_counts().index[0]))

In [ ]:
train['CREDIT_INCOME_PERCENT'] = train['AMT_CREDIT'] / train['AMT_INCOME_TOTAL']
train['ANNUITY_INCOME_PERCENT'] = train['AMT_ANNUITY'] / train['AMT_INCOME_TOTAL']
train['CREDIT_TERM'] = train['AMT_ANNUITY'] / train['AMT_CREDIT']
train['DAYS_EMPLOYED_PERCENT'] = train['DAYS_EMPLOYED'] / train['DAYS_BIRTH']
train['INCOME_PER_PERSON'] = train['AMT_INCOME_TOTAL'] / train['CNT_FAM_MEMBERS']
train['ANNUITY_INCOME_PERC'] = train['AMT_ANNUITY'] / train['AMT_INCOME_TOTAL']

In [ ]:
train.shape

(307511, 128)

## POS_CASH

In [ ]:
description.loc[description.Table=='POS_CASH_balance.csv']

,Table,Row,Description,Special
145,POS_CASH_balance.csv,SK_ID_PREV,ID of previous credit in Home Credit related t...,NaN
146,POS_CASH_balance.csv,SK_ID_CURR,ID of loan in our sample,NaN
147,POS_CASH_balance.csv,MONTHS_BALANCE,Month of balance relative to application date ...,time only relative to the application
148,POS_CASH_balance.csv,CNT_INSTALMENT,Term of previous credit (can change over time),NaN
149,POS_CASH_balance.csv,CNT_INSTALMENT_FUTURE,Installments left to pay on the previous credit,NaN
150,POS_CASH_balance.csv,NAME_CONTRACT_STATUS,Contract status during the month,NaN
151,POS_CASH_balance.csv,SK_DPD,DPD (days past due) during the month of previo...,NaN
152,POS_CASH_balance.csv,SK_DPD_DEF,DPD during the month with tolerance (debts wit...,NaN


In [ ]:
pos_cash = pd.read_csv('/content/drive/My Drive/HomeCredit/POS_CASH_balance.csv')

In [ ]:
pos_cash.isna().sum()

SK_ID_PREV                   0
SK_ID_CURR                   0
MONTHS_BALANCE               0
CNT_INSTALMENT           26071
CNT_INSTALMENT_FUTURE    26087
NAME_CONTRACT_STATUS         0
SK_DPD                       0
SK_DPD_DEF                   0
dtype: int64

In [ ]:
cat_cols = [col for col in pos_cash.columns if pos_cash[col].dtype == 'object']
pos_cash = pd.get_dummies(pos_cash, columns= cat_cols, dummy_na= True)
aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size', 'std'],
        'SK_DPD': ['max', 'mean', 'std'],
        'SK_DPD_DEF': ['max', 'mean', 'std']
}
name_cols = pos_cash.columns[7:]
for col in name_cols : aggregations[col] = ['mean'] 

pos_cash_agg = pos_cash.groupby('SK_ID_CURR').agg(aggregations)
pos_cash_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_cash_agg.columns.tolist()]) #переназвать колонки
pos_cash_agg['PREV_CRED_COUNT'] = pos_cash.groupby('SK_ID_CURR').size() # количество взятых ранее кредитов 

In [ ]:
pos_cash_agg

,POS_MONTHS_BALANCE_MAX,POS_MONTHS_BALANCE_MEAN,POS_MONTHS_BALANCE_SIZE,POS_MONTHS_BALANCE_STD,POS_SK_DPD_MAX,POS_SK_DPD_MEAN,POS_SK_DPD_STD,POS_SK_DPD_DEF_MAX,POS_SK_DPD_DEF_MEAN,POS_SK_DPD_DEF_STD,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Amortized debt_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_nan_MEAN,PREV_CRED_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-53,-72.555556,9,20.863312,7,0.777778,2.333333,7,0.777778,2.333333,0.777778,0.0,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0,9
100002,-1,-10.000000,19,5.627314,0,0.000000,0.000000,0,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0,19
100003,-18,-43.785714,28,24.640162,0,0.000000,0.000000,0,0.000000,0.000000,0.928571,0.0,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0,28
100004,-24,-25.500000,4,1.290994,0,0.000000,0.000000,0,0.000000,0.000000,0.750000,0.0,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0,4
100005,-15,-20.000000,11,3.316625,0,0.000000,0.000000,0,0.000000,0.000000,0.818182,0.0,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,-1,-5.000000,9,2.738613,0,0.000000,0.000000,0,0.000000,0.000000,0.777778,0.0,0.0,0.0,0.111111,0.0,0.0,0.111111,0.0,0,9
456252,-76,-79.000000,7,2.160247,0,0.000000,0.000000,0,0.000000,0.000000,0.857143,0.0,0.0,0.0,0.142857,0.0,0.0,0.000000,0.0,0,7
456253,-57,-79.235294,17,16.723970,5,0.294118,1.212678,5,0.294118,1.212678,0.882353,0.0,0.0,0.0,0.117647,0.0,0.0,0.000000,0.0,0,17


In [ ]:
# pos_cash_agg.to_csv('/content/drive/My Drive/HomeCredit/POS_CAHS_agg.csv')
train = pd.merge(train, pos_cash_agg, on='SK_ID_CURR', how='left')

In [ ]:
train.shape

(307511, 149)

## PREVIOUS APPLICATIONS 



In [ ]:
description.loc[description.Table=='previous_application.csv']

,Table,Row,Description,Special
176,previous_application.csv,SK_ID_PREV,ID of previous credit in Home credit related t...,hashed
177,previous_application.csv,SK_ID_CURR,ID of loan in our sample,hashed
178,previous_application.csv,NAME_CONTRACT_TYPE,"Contract product type (Cash loan, consumer loa...",NaN
179,previous_application.csv,AMT_ANNUITY,Annuity of previous application,NaN
180,previous_application.csv,AMT_APPLICATION,For how much credit did client ask on the prev...,NaN
181,previous_application.csv,AMT_CREDIT,Final credit amount on the previous applicatio...,NaN
182,previous_application.csv,AMT_DOWN_PAYMENT,Down payment on the previous application,NaN
183,previous_application.csv,AMT_GOODS_PRICE,Goods price of good that client asked for (if ...,NaN
184,previous_application.csv,WEEKDAY_APPR_PROCESS_START,On which day of the week did the client apply ...,NaN
185,previous_application.csv,HOUR_APPR_PROCESS_START,Approximately at what day hour did the client ...,rounded


In [ ]:
prev_app = pd.read_csv('/content/drive/My Drive/HomeCredit/previous_application.csv')

In [ ]:
original_columns = list(prev_app.columns) #One Hot
cat_cols = [col for col in prev_app.columns if prev_app[col].dtype == 'object']
prev_app = pd.get_dummies(prev_app, columns= cat_cols, dummy_na= True)
onehot_cols = [col for col in prev_app.columns if col not in original_columns] # Новые колонки чтобы агрегировать по ним

In [ ]:
prev_app['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True) #пофиксить шум
prev_app['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
prev_app['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
prev_app['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
prev_app['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
prev_app['APP_CREDIT_PERC'] = prev_app['AMT_APPLICATION'] / prev_app['AMT_CREDIT']


aggregations = {
  'AMT_ANNUITY': ['min', 'max', 'mean'],
  'AMT_APPLICATION': ['min', 'max', 'mean'],
  'AMT_CREDIT': ['min', 'max', 'mean'],
  'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
  'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
  'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
  'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
  'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
  'DAYS_DECISION': ['min', 'max', 'mean'],
  'CNT_PAYMENT': ['mean', 'sum'],
}
aggregations = {}
for col in onehot_cols: aggregations[col] = ['mean']

prev_app_agg = prev_app.groupby('SK_ID_CURR').agg(aggregations)
prev_app_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_app_agg.columns.tolist()]) #переназвать колонки, чтобы было понятно 
# Previous Applications: Approved Applications - only numerical features
# approved = prev_app[prev_app['NAME_CONTRACT_STATUS_Approved'] == 1]
# approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
# approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
# prev_app_agg = prev_app_agg.join(approved_agg, how='left', on='SK_ID_CURR')
# # Previous Applications: Refused Applications - only numerical features
# refused = prev_app[prev_app['NAME_CONTRACT_STATUS_Refused'] == 1]
# refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
# refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
# prev_app_agg = prev_app_agg.join(refused_agg, how='left', on='SK_ID_CURR')

In [ ]:
prev_app_agg

,PREV_NAME_CONTRACT_TYPE_Cash loans_MEAN,PREV_NAME_CONTRACT_TYPE_Consumer loans_MEAN,PREV_NAME_CONTRACT_TYPE_Revolving loans_MEAN,PREV_NAME_CONTRACT_TYPE_XNA_MEAN,PREV_NAME_CONTRACT_TYPE_nan_MEAN,PREV_WEEKDAY_APPR_PROCESS_START_FRIDAY_MEAN,PREV_WEEKDAY_APPR_PROCESS_START_MONDAY_MEAN,PREV_WEEKDAY_APPR_PROCESS_START_SATURDAY_MEAN,PREV_WEEKDAY_APPR_PROCESS_START_SUNDAY_MEAN,PREV_WEEKDAY_APPR_PROCESS_START_THURSDAY_MEAN,PREV_WEEKDAY_APPR_PROCESS_START_TUESDAY_MEAN,PREV_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_MEAN,PREV_WEEKDAY_APPR_PROCESS_START_nan_MEAN,PREV_FLAG_LAST_APPL_PER_CONTRACT_N_MEAN,PREV_FLAG_LAST_APPL_PER_CONTRACT_Y_MEAN,PREV_FLAG_LAST_APPL_PER_CONTRACT_nan_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Building a house or an annex_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Business development_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Buying a garage_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Buying a holiday home / land_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Buying a home_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Buying a new car_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Buying a used car_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Car repairs_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Education_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Everyday expenses_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Furniture_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Gasification / water supply_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Hobby_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Journey_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Medicine_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Money for a third person_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Other_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Payments on other loans_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Purchase of electronic equipment_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Refusal to name the goal_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Repairs_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Urgent needs_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Wedding / gift / holiday_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_XAP_MEAN,...,PREV_CHANNEL_TYPE_Credit and cash offices_MEAN,PREV_CHANNEL_TYPE_Regional / Local_MEAN,PREV_CHANNEL_TYPE_Stone_MEAN,PREV_CHANNEL_TYPE_nan_MEAN,PREV_NAME_SELLER_INDUSTRY_Auto technology_MEAN,PREV_NAME_SELLER_INDUSTRY_Clothing_MEAN,PREV_NAME_SELLER_INDUSTRY_Connectivity_MEAN,PREV_NAME_SELLER_INDUSTRY_Construction_MEAN,PREV_NAME_SELLER_INDUSTRY_Consumer electronics_MEAN,PREV_NAME_SELLER_INDUSTRY_Furniture_MEAN,PREV_NAME_SELLER_INDUSTRY_Industry_MEAN,PREV_NAME_SELLER_INDUSTRY_Jewelry_MEAN,PREV_NAME_SELLER_INDUSTRY_MLM partners_MEAN,PREV_NAME_SELLER_INDUSTRY_Tourism_MEAN,PREV_NAME_SELLER_INDUSTRY_XNA_MEAN,PREV_NAME_SELLER_INDUSTRY_nan_MEAN,PREV_NAME_YIELD_GROUP_XNA_MEAN,PREV_NAME_YIELD_GROUP_high_MEAN,PREV_NAME_YIELD_GROUP_low_action_MEAN,PREV_NAME_YIELD_GROUP_low_normal_MEAN,PREV_NAME_YIELD_GROUP_middle_MEAN,PREV_NAME_YIELD_GROUP_nan_MEAN,PREV_PRODUCT_COMBINATION_Card Street_MEAN,PREV_PRODUCT_COMBINATION_Card X-Sell_MEAN,PREV_PRODUCT_COMBINATION_Cash_MEAN,PREV_PRODUCT_COMBINATION_Cash Street: high_MEAN,PREV_PRODUCT_COMBINATION_Cash Street: low_MEAN,PREV_PRODUCT_COMBINATION_Cash Street: middle_MEAN,PREV_PRODUCT_COMBINATION_Cash X-Sell: high_MEAN,PREV_PRODUCT_COMBINATION_Cash X-Sell: low_MEAN,PREV_PRODUCT_COMBINATION_Cash X-Sell: middle_MEAN,PREV_PRODUCT_COMBINATION_POS household with interest_MEAN,PREV_PRODUCT_COMBINATION_POS household without interest_MEAN,PREV_PRODUCT_COMBINATION_POS industry with interest_MEAN,PREV_PRODUCT_COMBINATION_POS industry without interest_MEAN,PREV_PRODUCT_COMBINATION_POS mobile with interest_MEAN,PREV_PRODUCT_COMBINATION_POS mobile without interest_MEAN,PREV_PRODUCT_COMBINATION_POS other with interest_MEAN,PREV_PRODUCT_COMBINATION_POS others without interest_MEAN,PREV_PRODUCT_COMBINATION_nan_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,0.000000,1.000000,0.000,0.0,0,1.000000,0.0,0.000000,0.000000,0.00,0.000,0.0,0,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,1.000000,...,0.000000,0.000,0.000000,0,0.0,0.0,1.000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.

In [ ]:
# prev_app_agg.to_csv('/content/drive/My Drive/HomeCredit/previous_applications_agg.csv')
train = pd.merge(train, prev_app_agg, on='SK_ID_CURR', how='left')

In [ ]:
train.shape

(307511, 308)

## BUREU

In [ ]:
def one_hot_encoder(df):
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    return pd.get_dummies(df, columns=categorical_columns, dummy_na=True)

In [ ]:
bureau = pd.read_csv('/content/drive/My Drive/HomeCredit/bureau.csv')
cnt_active_loans = bureau[bureau.CREDIT_ACTIVE == 'Active'].groupby('SK_ID_CURR', as_index=False)['CREDIT_ACTIVE'].count()
cnt_closed_loans = bureau[bureau.CREDIT_ACTIVE == 'Closed'].groupby('SK_ID_CURR', as_index=False)['CREDIT_ACTIVE'].count()
# Мерджим фичи

train = train.merge(cnt_active_loans, on = 'SK_ID_CURR', how = 'left').fillna(0)
train = train.merge(cnt_closed_loans, on = 'SK_ID_CURR', how = 'left').fillna(0)

cnt_numeric_fields_statistics = bureau.drop(columns = ['SK_ID_BUREAU']).groupby('SK_ID_CURR', as_index = False).agg(['min', 'max', 'mean', 'sum'])
cnt_numeric_fields_statistics.reset_index(inplace=True)

columns = ['SK_ID_CURR']
for column in cnt_numeric_fields_statistics.columns.levels[0]:
    if column != 'SK_ID_CURR':
        for stat in cnt_numeric_fields_statistics.columns.levels[1][:-1]:
            columns.append('bureau_%s_%s' % (column, stat))

cnt_numeric_fields_statistics.columns = columns

train = train.merge(cnt_numeric_fields_statistics, on = 'SK_ID_CURR', how = 'left').fillna(0)

credit_type_ohe = one_hot_encoder(bureau)

credit_type_ohe = credit_type_ohe.groupby('SK_ID_CURR', as_index=False).count()

train = train.merge(credit_type_ohe, on = 'SK_ID_CURR', how = 'left').fillna(0)

In [ ]:
train.shape

(307511, 397)

## INSTALLMENTS PAYMENTS и CREDIT CARD BALANCE

In [ ]:
IP = pd.read_csv('/content/drive/My Drive/HomeCredit/installments_payments.csv')
BadSKID = list(IP[IP['DAYS_ENTRY_PAYMENT'].isna()]['SK_ID_CURR'])
new = IP[['SK_ID_CURR', 'AMT_INSTALMENT', 'AMT_PAYMENT', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT']]

def f(x):
  if x['SK_ID_CURR'] in BadSKID:
    x['IN_BLACK_LIST'] = 'bad'
  return x


del IP
new = new.dropna()
new = new.groupby(['SK_ID_CURR'], as_index=False).sum()
new['IN_BLACK_LIST'] = 'good'
new['DAYS_GONE'] = new['DAYS_ENTRY_PAYMENT'] - new['DAYS_INSTALMENT']
new = new.apply(f, axis=1)
new = new.drop(columns=['DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT'])

credit_card_balance = pd.read_csv('/content/drive/My Drive/HomeCredit/credit_card_balance.csv')
cc = credit_card_balance[['SK_ID_CURR', 'SK_DPD', 'SK_DPD_DEF']]
cc = cc.groupby(['SK_ID_CURR'], as_index=False).sum()
new_ohe = one_hot_encoder(new)
values = {'SK_DPD': 0, 'SK_DPD_DEF': 0, 'AMT_INSTALMENT': 0, 'AMT_PAYMENT': 0, 'DAYS_GONE': 0}
train = pd.merge(train, cc, how='left', on='SK_ID_CURR')
train = pd.merge(train, new_ohe, how='left', on='SK_ID_CURR')
train = train.fillna(value=values)

In [ ]:
train.shape

(307511, 405)

## BUREAU_BALANCE

In [ ]:
bureau_balance = pd.read_csv('/content/drive/My Drive/HomeCredit/bureau_balance.csv')
bureau_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [ ]:
def aggregate_numeric_fields(df, columns_to_drop, group_by, agg_methods, name_prefix):
    cnt_numeric_fields_statistics = df.drop(columns = columns_to_drop).groupby(group_by, as_index = False).agg(agg_methods).reset_index()

    columns = [group_by]
    for column in cnt_numeric_fields_statistics.columns.levels[0]:
        if column == group_by:
            continue

        for stat in cnt_numeric_fields_statistics.columns.levels[1][:-1]:
            columns.append('{}_{}_{}'.format(name_prefix, column, stat))

    cnt_numeric_fields_statistics.columns = columns

    return cnt_numeric_fields_statistics

In [ ]:
cnt_numeric_fields_statistics = aggregate_numeric_fields(bureau_balance, ['STATUS'], 'SK_ID_BUREAU', ['min', 'mean', 'sum'], 'balance')
status_ohe = one_hot_encoder(bureau_balance)

# И статистику по этому кодированию.

status_ohe_statistics = aggregate_numeric_fields(status_ohe, [], 'SK_ID_BUREAU', ['min', 'max', 'mean', 'sum'], 'balance')

bureau_balance_features = cnt_numeric_fields_statistics.merge(status_ohe_statistics, on = 'SK_ID_BUREAU', how = 'left').fillna(0)
bureau_X_bureau_balance_features = bureau[['SK_ID_CURR', 'SK_ID_BUREAU']].merge(bureau_balance_features, on = 'SK_ID_BUREAU', how = 'left').fillna(0)

# И опять статистику.

bureau_X_bureau_balance_features_features = aggregate_numeric_fields(bureau_X_bureau_balance_features, ['SK_ID_BUREAU'], 'SK_ID_CURR', ['min', 'max', 'mean', 'sum'], 'id')
train = train.merge(bureau_X_bureau_balance_features_features, on = 'SK_ID_CURR', how = 'left').fillna(0)

In [ ]:
train = one_hot_encoder(train)

In [ ]:
train.shape

(307511, 718)

In [ ]:
for col in train.columns: 
  if len(train[col].unique()) == 1:
    train.drop(columns=[col], inplace=True)


# при расчете корреляция появлялись Nanы - из за отсутствия дисперсии в колонках

In [ ]:
train.shape

(307511, 656)

In [ ]:
train.to_csv('/content/drive/My Drive/HomeCredit/train.csv')